In [1]:
import requests
import datetime
import time
import csv

def fetch_binance_data(symbol, interval, start_time, end_time):
    BASE_URL = 'https://fapi.binance.com/fapi/v1/klines'
    limit = 1000  # maximum number of results per request
    all_klines = []

    while start_time < end_time:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': limit
        }
        response = requests.get(BASE_URL, params=params)
        klines = response.json()
        if not klines:
            break
        all_klines.extend(klines)
        start_time = klines[-1][0] + 1  # start where the last batch left off
        time.sleep(1)  # wait for a second before making another request

    return all_klines

def save_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['time', 'open', 'high', 'low', 'close', 'volume'])
        for entry in data:
            time_str = datetime.datetime.fromtimestamp(entry[0] / 1000).strftime('%Y-%m-%d %H:%M:%S')
            writer.writerow([time_str, entry[1], entry[2], entry[3], entry[4], entry[5]])

# Parameters
symbol = 'THETAUSDT'
interval = '1h'
end_time = int(time.time() * 1000)  # current time in milliseconds
start_time = end_time - (1080 * 24 * 60 * 60 * 1000)  

data = fetch_binance_data(symbol, interval, start_time, end_time)

# Save data to CSV
save_to_csv(data, 'THETA.csv')

ConnectionError: HTTPSConnectionPool(host='fapi.binance.com', port=443): Max retries exceeded with url: /fapi/v1/klines?symbol=THETAUSDT&interval=1h&startTime=1604914860130&endTime=1698226860130&limit=1000 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x10533fd60>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))